In [1]:
import random
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

class SensorNeuron(nn.Module):
    def __init__(self, input_size, dataset_samples, threshold=0.5):
        super(SensorNeuron, self).__init__()
        self.input_size = input_size
        self.dataset_samples = dataset_samples
        self.threshold = threshold  # Limiar para sinal forte
        self.sensory_db = self.initialize_sensory_neurons()  # Banco de dados sensorial
        self.consolidation_counter = [0] * input_size  # Contadores de consolidação
    
    def initialize_sensory_neurons(self):
        sensory_neurons = []
        for _ in range(self.input_size):
            random_sample = random.choice(self.dataset_samples)
            sensory_neurons.append(random.choice(random_sample))  # Neurônios inicializados com dados aleatórios
        return torch.tensor(sensory_neurons).float()
    
    def forward(self, x):
        x = x.float()  # Garantir tipo correto de entrada
        reinforcement_score = 0  # Inicia o score de reforço
        
        # Ordenar os índices dos neurônios de acordo com a consolidação (priorizando os reforçados)
        sorted_indices = sorted(range(self.input_size), key=lambda i: self.consolidation_counter[i], reverse=True)
        
        for i in sorted_indices:  # Processa os neurônios reforçados primeiro
            # Calcula a diferença absoluta
            diff = torch.abs(x[:, i] - self.sensory_db[i])
            
            # Verifica se cada elemento no lote é menor que o threshold
            signal_strong = diff < self.threshold
            
            # Verifica se a maioria dos sinais é forte (True)
            if signal_strong.all():  # Se todos os valores no lote são menores que o limiar
                self.consolidation_counter[i] += 1
                reinforcement_score += 1  # Sinal forte encontrado, reforça a informação
            else:
                self.consolidation_counter[i] -= 1  # Sinal fraco, reduz a consolidação
            
            # Caso o contador de consolidação seja negativo, atualiza o neurônio
            if self.consolidation_counter[i] < 0:
                self.sensory_db = self.update_sensory_neuron(self.sensory_db, self.dataset_samples, i)  # Chama a função externa
        
        return x, reinforcement_score
    
    def update_sensory_neuron(self, sensory_db, dataset_samples, index):
        random_sample = random.choice(dataset_samples)
        new_value = random.choice(random_sample)
        sensory_db[index] = new_value
        return sensory_db
        
    def reinforce_database(self):
        # Caso o sinal seja encontrado mais de uma vez, a informação é consolidada no banco de dados permanente
        for i in range(self.input_size):
            if self.consolidation_counter[i] > 5:  # Exemplo de limiar de consolidação
                self.sensory_db[i] = self.sensory_db[i]  # Reforço da informação
            elif self.consolidation_counter[i] < -3:
                self.update_sensory_neuron(i)  # Atualiza os neurônios com base no contador de consolidação


class AdvancedMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dataset_samples, threshold=0.5):
        super(AdvancedMLP, self).__init__()
        self.sensor_neuron = SensorNeuron(input_size=input_size, dataset_samples=dataset_samples, threshold=threshold)  # Neurônios sensoriais
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x, reinforcement_score = self.sensor_neuron(x)  # Processamento inicial com neurônios sensoriais
        x = F.relu(self.fc1(x))  # Primeira camada de rede neural
        x = self.fc2(x)  # Camada final de saída
        return F.log_softmax(x, dim=1), reinforcement_score





In [3]:
# Carregar o dataset Iris
iris = load_iris()
X = iris.data  # Características (features)
y = iris.target  # Rótulos (labels)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores do PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Criar DataLoader para treino
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Criar DataLoader para teste
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o modelo
input_size = X_train.shape[1]  # 4 características de entrada
hidden_size = 16
output_size = len(set(y))  # 3 classes (Setosa, Versicolor, Virginica)
dataset_samples = [list(range(10))]  # Exemplo simples, substitua conforme necessário
threshold = 0.5

model = AdvancedMLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dataset_samples=dataset_samples, threshold=threshold)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Para classificação
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_samples = 0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs, reinforcement_score = model(inputs)
        
        # Calcular a perda
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular acurácia
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_preds / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Avaliar o modelo nos dados de teste
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs, _ = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * correct_preds / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch [1/200], Loss: 1.1869, Accuracy: 23.33%
Epoch [2/200], Loss: 1.1660, Accuracy: 23.33%
Epoch [3/200], Loss: 1.1474, Accuracy: 23.33%
Epoch [4/200], Loss: 1.1278, Accuracy: 23.33%
Epoch [5/200], Loss: 1.1114, Accuracy: 23.33%
Epoch [6/200], Loss: 1.0921, Accuracy: 23.33%
Epoch [7/200], Loss: 1.0749, Accuracy: 31.67%
Epoch [8/200], Loss: 1.0561, Accuracy: 40.83%
Epoch [9/200], Loss: 1.0386, Accuracy: 44.17%
Epoch [10/200], Loss: 1.0198, Accuracy: 48.33%
Epoch [11/200], Loss: 0.9998, Accuracy: 50.83%
Epoch [12/200], Loss: 0.9817, Accuracy: 51.67%
Epoch [13/200], Loss: 0.9633, Accuracy: 53.33%
Epoch [14/200], Loss: 0.9449, Accuracy: 52.50%
Epoch [15/200], Loss: 0.9273, Accuracy: 52.50%
Epoch [16/200], Loss: 0.9054, Accuracy: 59.17%
Epoch [17/200], Loss: 0.8899, Accuracy: 64.17%
Epoch [18/200], Loss: 0.8678, Accuracy: 70.83%
Epoch [19/200], Loss: 0.8526, Accuracy: 74.17%
Epoch [20/200], Loss: 0.8348, Accuracy: 75.83%
Epoch [21/200], Loss: 0.8226, Accuracy: 75.83%
Epoch [22/200], Loss: 

In [4]:
# Carregar o dataset digits
digits = load_digits()
X = digits.data  # Características (features)
y = digits.target  # Rótulos (labels)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores do PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Criar DataLoader para treino
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Criar DataLoader para teste
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o modelo
input_size = X_train.shape[1]  # 4 características de entrada
hidden_size = 16
output_size = len(set(y))  # 3 classes (Setosa, Versicolor, Virginica)
dataset_samples = [list(range(10))]  # Exemplo simples, substitua conforme necessário
threshold = 0.5

model = AdvancedMLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dataset_samples=dataset_samples, threshold=threshold)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Para classificação
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_samples = 0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs, reinforcement_score = model(inputs)
        
        # Calcular a perda
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular acurácia
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_preds / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Avaliar o modelo nos dados de teste
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs, _ = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * correct_preds / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")

Epoch [1/100], Loss: 2.0505, Accuracy: 39.39%
Epoch [2/100], Loss: 1.6215, Accuracy: 64.23%
Epoch [3/100], Loss: 1.2095, Accuracy: 76.13%
Epoch [4/100], Loss: 0.8808, Accuracy: 83.51%
Epoch [5/100], Loss: 0.6603, Accuracy: 88.03%
Epoch [6/100], Loss: 0.5127, Accuracy: 90.61%
Epoch [7/100], Loss: 0.4131, Accuracy: 91.65%
Epoch [8/100], Loss: 0.3439, Accuracy: 93.18%
Epoch [9/100], Loss: 0.2926, Accuracy: 94.36%
Epoch [10/100], Loss: 0.2548, Accuracy: 95.06%
Epoch [11/100], Loss: 0.2242, Accuracy: 95.27%
Epoch [12/100], Loss: 0.2002, Accuracy: 95.69%
Epoch [13/100], Loss: 0.1800, Accuracy: 96.17%
Epoch [14/100], Loss: 0.1631, Accuracy: 97.15%
Epoch [15/100], Loss: 0.1489, Accuracy: 97.56%
Epoch [16/100], Loss: 0.1370, Accuracy: 97.63%
Epoch [17/100], Loss: 0.1264, Accuracy: 98.05%
Epoch [18/100], Loss: 0.1174, Accuracy: 98.12%
Epoch [19/100], Loss: 0.1089, Accuracy: 98.26%
Epoch [20/100], Loss: 0.1018, Accuracy: 98.40%
Epoch [21/100], Loss: 0.0955, Accuracy: 98.47%
Epoch [22/100], Loss: 